# LangGraph Intro

Here we follow [this tutorial](https://github.com/LangGraph-GUI/LangGraph-learn/tree/main/02%20LangGraph%20Hello%20World), [this tutorial](https://github.com/LangGraph-GUI/LangGraph-learn/tree/main/03%20Graph%20Chain%20Hello%20World), and [this tutorial](https://github.com/LangGraph-GUI/LangGraph-learn/tree/main/04%20State%20Graph).

## Prerequisites

1. `ollama` server
2. `langchain-ollama`
3. `langchain-core`
4. `langgraph`

## Building a Simple Workflow without LLMs 

In [this demo](https://github.com/LangGraph-GUI/LangGraph-learn/tree/main/02%20LangGraph%20Hello%20World), two nodes in a chained fashion will cooperate to construct a greeting string.

In [1]:
from langgraph.graph import Graph

In [2]:
# define two simple functions to be used in the workflow
def func_1(x1):
    return f"Hello {x1}."

def func_2(x2):
    return f"{x2} How are you?"

In [3]:
NODE_1_NAME = 'node_1'
NODE_2_NAME = 'node_2'

In [4]:
# create a graph where each function is a node
workflow = Graph()
workflow.add_node(NODE_1_NAME, func_1)
workflow.add_node(NODE_2_NAME, func_2)
# also define edges to connect the nodes
workflow.add_edge(NODE_1_NAME, NODE_2_NAME)
# finally set the entry and finish points of the workflow
workflow.set_entry_point(NODE_1_NAME)
_ = workflow.set_finish_point(NODE_2_NAME)

Now we have a workflow as a graph: START --> node_1 --> node_2 --> END.

In [5]:
# compile the workflow
app = workflow.compile()

In [6]:
# invoke the compiled workflow
res = app.invoke("Mike")
print(res)

Hello Mike. How are you?


## Building a Workflow that Utilizes an LLM

In [this demo](https://github.com/LangGraph-GUI/LangGraph-learn/tree/main/03%20Graph%20Chain%20Hello%20World), an agent will accept the user's request to write a story into a local file.

In [7]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

In [8]:
# initialize the ChatOllama model with desired parameters
# https://pypi.org/project/langchain-ollama/
# https://python.langchain.com/docs/integrations/chat/ollama/
# https://python.langchain.com/api_reference/ollama/chat_models/langchain_ollama.chat_models.ChatOllama.html
llm = ChatOllama(model="qwen2.5:7b", format="json")

In [9]:
# define a prompt template for the agent
template = """
    You are an excellent writer capable of producing amazing stories and analyses.
    Question: {question}
    On the top level, the output should contain a "filename" JSON key to capture any file name provided by the user. It should also contain a "content" JSON key to place your generation result.
    If the user does not provide a file name, try to randomly generate one for the user.
    The format of the "content" JSON key's value depends on the user's request. By default, it should be a single string of Markdown style.
"""
prompt = PromptTemplate.from_template(template)

In [10]:
# define a writer agent to output strict json text
WRITER_NAME = 'writer'
def Writer(question):
    formatted_prompt = prompt.format(question=question)
    print(f'{WRITER_NAME}: Writing...')
    generation = llm.invoke(formatted_prompt)
    print(f'{WRITER_NAME}: Finished Writing.')
    return generation.content

In [11]:
import json
import os

OUT_DIR = 'out/'
os.makedirs(OUT_DIR, exist_ok=True)

# define a dumper to get LLM's response JSON and actually store the generated text to a file
DUMPER_NAME = 'dumper'
def Dumper(result_json):
    print(f'{DUMPER_NAME}: Loading JSON...')
    data = json.loads(result_json)
    print(f'{DUMPER_NAME}: JSON loaded successfully.')
    # extract outputs
    content = data.get('content', "")
    filename = data.get('filename', "output.md")
    # write
    print(f'{DUMPER_NAME}: Writing to {filename}...')
    with open(os.path.join(f'out/{filename}'), 'w') as file:
        file.write(content)
    print(f'{DUMPER_NAME}: Finished writing.')
    return data

In [12]:
# create a graph where each function is a node
workflow = Graph()
workflow.add_node(WRITER_NAME, Writer)
workflow.add_node(DUMPER_NAME, Dumper)
# also define edges to connect the nodes
workflow.add_edge(WRITER_NAME, DUMPER_NAME)
# finally set the entry and finish points of the workflow
workflow.set_entry_point(WRITER_NAME)
_ = workflow.set_finish_point(DUMPER_NAME)

Now test~

In [13]:
# compile the workflow
app = workflow.compile()

In [14]:
# invoke the compiled workflow
def run(prompt):
    res = app.invoke(prompt)
    print(f'> JSON dict keys: {res.keys()}')
    print(f'> File name: {res["filename"]}')
    print(res['content'])

In [15]:
run('Suggest how to wake up early in the morning - in a file named "wakeup.md".')

writer: Writing...
writer: Finished Writing.
dumper: Loading JSON...
dumper: JSON loaded successfully.
dumper: Writing to wakeup.md...
dumper: Finished writing.
> JSON dict keys: dict_keys(['filename', 'content'])
> File name: wakeup.md
# How to Wake Up Early in the Morning

## Introduction
Waking up early can have numerous benefits such as better productivity and stress management. Here are some tips on how to establish an early morning routine.

## Establish a Consistent Sleep Schedule
Try to go to bed and wake up at the same time every day, even on weekends. This will help regulate your body's internal clock and make waking up easier.

## Create a Bedtime Routine
Engage in calming activities before bedtime such as reading or listening to soothing music. Avoid screens an hour before bed to help you wind down more effectively.

## Use Alarms Strategically
Set multiple alarms at progressively earlier times, but avoid setting too many as it can be overwhelming and lead to frustration. P

In [16]:
run('What is the ultimate goal of life?')

writer: Writing...
writer: Finished Writing.
dumper: Loading JSON...
dumper: JSON loaded successfully.
dumper: Writing to the_ultimate_goal_of_life.md...
dumper: Finished writing.
> JSON dict keys: dict_keys(['filename', 'content'])
> File name: the_ultimate_goal_of_life.md
# The Ultimate Goal of Life: A Quest for Meaning and Fulfillment

The ultimate goal of life is a deeply personal and subjective question that has puzzled philosophers, theologians, and thinkers throughout history. While there isn't one single answer that applies to everyone, many individuals seek meaning and fulfillment through a combination of internal and external factors.

## The Search for Purpose

At the core of finding one's ultimate goal lies the search for purpose. This can manifest in various ways, such as:

- **Personal Development**: Continuously growing and improving oneself to reach one’s full potential.
- **Service**: Contributing to others or society, whether through volunteering, mentoring, or other 

In [17]:
run('In Python, how to build a simple merge sort program?')

writer: Writing...
writer: Finished Writing.
dumper: Loading JSON...
dumper: JSON loaded successfully.
dumper: Writing to merge_sort_program.py...
dumper: Finished writing.
> JSON dict keys: dict_keys(['filename', 'content'])
> File name: merge_sort_program.py
# Merge Sort in Python

## Introduction

Merge sort is a divide-and-conquer algorithm that divides an array into halves, sorts each half, and then merges them back together. This example demonstrates how to implement merge sort in Python.

## Code Implementation
```python
def merge_sort(arr):
    if len(arr) > 1:
        mid = len(arr) // 2
        left_half = arr[:mid]
        right_half = arr[mid:]

        # Recursive call on each half
        merge_sort(left_half)
        merge_sort(right_half)

        # Two iterators for traversing the two halves
        i = j = k = 0

        # Copy data to temp arrays left_half and right_half
        while i < len(left_half) and j < len(right_half):
            if left_half[i] < right_hal

## Building a State Machine (No LLM Integration)

In [this demo](https://github.com/LangGraph-GUI/LangGraph-learn/tree/main/04%20State%20Graph), a buyer will keep buying lottery round by round until he wins.

In [18]:
from langgraph.graph import StateGraph, END
from typing import TypedDict
import random
from pprint import pprint

In [19]:
# State Representation
class LotteryState(TypedDict):
    buy_num: int
    prize_num: int

In [20]:
# Buy Lottery Node
BUYER_NODE_NAME = 'buyer'
def Buyer(state: LotteryState):
    rand_num = random.randint(0, 2) # 0, 1, 2
    print('-------------------------')
    print(f'\t> buy number: {rand_num}')
    state['buy_num'] = rand_num
    return state

In [21]:
# Prize Node
PRIZE_NODE_NAME = 'prize'
def Prize(state: LotteryState):
    prize_num = random.randint(0, 2) # 0, 1, 2
    print(f'\t> prize number: {prize_num}')
    state['prize_num'] = prize_num
    return state

In [22]:
# Checking Edge (Conditional)
def check_result(state: LotteryState):
    print(f'\t> State: {state}')
    if state['buy_num'] == state['prize_num']:
        print('\t> You win! Go home.')
        return END
    else:
        print('\t> You missed. Buy again.')
        return BUYER_NODE_NAME

- State Graph: official doc - [intro](https://langchain-ai.github.io/langgraph/concepts/low_level/#stategraph) and [reference](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.graph.CompiledGraph.get_graph).
- Conditional Edge: official doc - [reference](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.state.StateGraph.add_conditional_edges).

In [23]:
# create a graph
workflow = StateGraph(LotteryState)
workflow.add_node(BUYER_NODE_NAME, Buyer)
workflow.add_node(PRIZE_NODE_NAME, Prize)
workflow.set_entry_point(BUYER_NODE_NAME)
workflow.add_edge(BUYER_NODE_NAME, PRIZE_NODE_NAME)
_ = workflow.add_conditional_edges(PRIZE_NODE_NAME, check_result)

Now test~

In [24]:
# compile the workflow
app = workflow.compile()

Streaming: official doc - [intro](https://langchain-ai.github.io/langgraph/concepts/streaming/) and [reference](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.graph.CompiledGraph.stream).

In [25]:
# invoke the compiled workflow
for s in app.stream(LotteryState()):
    print(s)

-------------------------
	> buy number: 2
{'buyer': {'buy_num': 2}}
	> prize number: 1
	> State: {'buy_num': 2, 'prize_num': 1}
	> You missed. Buy again.
{'prize': {'buy_num': 2, 'prize_num': 1}}
-------------------------
	> buy number: 2
{'buyer': {'buy_num': 2, 'prize_num': 1}}
	> prize number: 2
	> State: {'buy_num': 2, 'prize_num': 2}
	> You win! Go home.
{'prize': {'buy_num': 2, 'prize_num': 2}}


## Nested Graph

In LangGraph, graphs can be nested together where nodes can be subgraphs. Check [this demo](https://github.com/LangGraph-GUI/LangGraph-learn/tree/main/08%20SubGraph) as an example. We will not demonstrate here.